In [1]:
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [2]:
df = pd.read_excel("../data/Chevron Challenge Materials/training.xlsx")

In [3]:
df.columns

Index(['Date', 'Vehicle Category', 'GVWR Class', 'Fuel Type', 'Model Year',
       'Fuel Technology', 'Electric Mile Range',
       'Number of Vehicles Registered at the Same Address', 'Region',
       'Vehicle Population'],
      dtype='object')

In [4]:
df.head()

,Date,Vehicle Category,GVWR Class,Fuel Type,Model Year,Fuel Technology,Electric Mile Range,Number of Vehicles Registered at the Same Address,Region,Vehicle Population
0,2019,P,Not Applicable,Gasoline,2020.0,ICE,Not Applicable,≥4,Statewide,395883
1,2020,P,Not Applicable,Gasoline,2020.0,ICE,Not Applicable,1,Statewide,370954
2,2021,P,Not Applicable,Gasoline,2020.0,ICE,Not Applicable,1,Statewide,349406
3,2019,P,Not Applicable,Gasoline,2019.0,ICE,Not Applicable,≥4,Statewide,348475
4,2019,P,Not Applicable,Gasoline,2018.0,ICE,Not Applicable,≥4,Statewide,333296


In [5]:
df.isna().any()

Date                                                 False
Vehicle Category                                     False
GVWR Class                                           False
Fuel Type                                            False
Model Year                                            True
Fuel Technology                                      False
Electric Mile Range                                  False
Number of Vehicles Registered at the Same Address    False
Region                                               False
Vehicle Population                                   False
dtype: bool

In [6]:
df[df['Model Year'].isna()]

,Date,Vehicle Category,GVWR Class,Fuel Type,Model Year,Fuel Technology,Electric Mile Range,Number of Vehicles Registered at the Same Address,Region,Vehicle Population
1323,2020,P,Not Applicable,Gasoline,NaN,ICE,Not Applicable,≥4,Statewide,22422
2076,2020,MC,Not Applicable,Gasoline,NaN,ICE,Not Applicable,≥4,Statewide,10516
2299,2020,P,Not Applicable,Gasoline,NaN,ICE,Not Applicable,3,Statewide,8658
2709,2020,P,Not Applicable,Gasoline,NaN,ICE,Not Applicable,2,Statewide,6604
3021,2020,P,Not Applicable,Gasoline,NaN,ICE,Not Applicable,1,Statewide,5506
...,...,...,...,...,...,...,...,...,...,...
40932,2019,T1,Unknown,Diesel,NaN,ICE,Not Applicable,3,Statewide,1
40980,2019,MH,Not Applicable,Diesel,NaN,ICE,Not Applicable,3,Statewide,1
40988,2019,MC,Not Applicable,Diesel,NaN,ICE,Not Applicable,2,Statewide,1
41044,2019,B,Not Applicable,Diesel,NaN,ICE,Not Applicable,2,Statewide,1


In [7]:
# drop regions as it is common
df = df.drop(['Region'],axis=1)

In [8]:
df['Model Year'] = df['Model Year'].astype('Int64')

In [9]:
gvwr_map = {
    '1': 'Light-Duty',
    '2': 'Light-Duty',
    '3': 'Medium-Duty',
    '4': 'Medium-Duty',
    '5': 'Medium-Duty',
    '6': 'Medium-Duty',
    '7': 'Heavy-Duty',
    '8': 'Heavy-Duty',
    'Not Applicable': 'Not Applicable',
    'Unknown': 'Unknown'
}

df['GVWR Group'] = df['GVWR Class'].map(gvwr_map)
df = pd.get_dummies(df, columns=['GVWR Group'], prefix='GVWR')

In [10]:
# Define mappings for ordinal encoding
mile_range_mapping = {
    'Not Applicable': 0,
    '0 to 50 miles': 1,
    '51 to 100 miles': 2,
    '101 to 150 miles': 3,
    '>150 miles': 4,
    'Unknown': -1
}

vehicle_count_mapping = {
    '1': 1,
    '2': 2,
    '3': 3,
    '≥4': 4,
    'Unknown': -1
}

# Apply ordinal encoding
df['Electric Mile Range'] = df['Electric Mile Range'].map(mile_range_mapping)
df['Number of Vehicles Registered at the Same Address'] = df['Number of Vehicles Registered at the Same Address'].map(vehicle_count_mapping)

In [11]:
df = pd.get_dummies(df, columns=['Vehicle Category', 'Fuel Type', 'Fuel Technology'], drop_first=True)

In [12]:
df = df.drop(['GVWR Class'],axis=1)

In [13]:
# attempt imputation technique to replace NaNs
imputer = IterativeImputer(random_state=100, max_iter=10)

In [14]:
imputer.fit(df)

IterativeImputer(random_state=100)

In [15]:
df_imputed = imputer.transform(df)

In [16]:
df_imputed

array([[2.019e+03, 2.020e+03, 0.000e+00, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [2.020e+03, 2.020e+03, 0.000e+00, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [2.021e+03, 2.020e+03, 0.000e+00, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       ...,
       [2.019e+03, 1.978e+03, 0.000e+00, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [2.019e+03, 1.976e+03, 0.000e+00, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [2.019e+03, 1.976e+03, 0.000e+00, ..., 0.000e+00, 1.000e+00,
        0.000e+00]], shape=(41053, 27))

In [17]:
df_ = pd.DataFrame(df_imputed)
df_.columns = df.columns 

In [18]:
df_.head()

,Date,Model Year,Electric Mile Range,Number of Vehicles Registered at the Same Address,Vehicle Population,GVWR_Not Applicable,GVWR_Unknown,Vehicle Category_BS,Vehicle Category_BT,Vehicle Category_MC,...,Vehicle Category_T6,Vehicle Category_T7,Fuel Type_Electric,Fuel Type_Gasoline,Fuel Type_Hydrogen,Fuel Type_Natural Gas,Fuel Type_Unknown,Fuel Technology_FCEV,Fuel Technology_ICE,Fuel Technology_PHEV
0,2019.0,2020.0,0.0,4.000000,395883.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2020.0,2020.0,0.0,4.121269,370954.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2021.0,2020.0,0.0,4.120229,349406.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,2019.0,2019.0,0.0,4.000000,348475.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,2019.0,2018.0,0.0,4.000000,333296.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [19]:
df_.shape

(41053, 27)

In [20]:
df_.isna().any()

Date                                                 False
Model Year                                           False
Electric Mile Range                                  False
Number of Vehicles Registered at the Same Address    False
Vehicle Population                                   False
GVWR_Not Applicable                                  False
GVWR_Unknown                                         False
Vehicle Category_BS                                  False
Vehicle Category_BT                                  False
Vehicle Category_MC                                  False
Vehicle Category_MH                                  False
Vehicle Category_P                                   False
Vehicle Category_T1                                  False
Vehicle Category_T2                                  False
Vehicle Category_T3                                  False
Vehicle Category_T4                                  False
Vehicle Category_T5                                  Fal

In [21]:
df_.to_csv("data/processed.csv")